# Introduction

The purpose of this notebook is to show some simple transformations that allow out of box non-parametric L2 regressors to achieve better SMAPE scores. By default, these algorithms are not optimizing for a better SMAPE score. However, by changing the target variables by an invertible transformation, we can get these algorithms to get closer to optimizing SMAPE. However a transformation alone won't make the algorithm actually optimize on SMAPE. Furthermore, fitting on transformed target variables will change a parametric model (e.g. linear regression), but a non-parametric model (e.g. decision tree or neural network) will be okay as it has no strict structure.

For this notebook, we will be using a simple decision tree for our model.

# Imports

Imports that we will need.

In [ ]:
import pandas as pd # Reading csv file
import numpy as np # Linear algebra
from matplotlib import pyplot as plt # Graphing
from sklearn.model_selection import train_test_split # We will do a simple train, validate, test split.
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# SMAPE Investigation

We won't go into great detail here as there are great resources elsewhere, but here we will just take a look at the nature of SMAPE. First we note that SMAPE measures an average of |p - t| / (|p| + |t|) for scalars p and t. Here, we think of p as being a predicted value and t being a true value. Furthermore, this quantity depends only on the ratio of p and t (with exception being when either are zero). So, let p = r * t. The quantity becomes a function of the single variable r, |1 - r| / (1 + |r|). Let us graph this function for non-negative ratios r.

In [ ]:
def ratioSMAPE(r):
    return np.abs(1 - r) / (1 + np.abs(r))

rvals = np.arange(0, 10, 0.1)
plt.plot(rvals, ratioSMAPE(rvals))
plt.title('Graph of |1 - r| / (1 + |r|)')
plt.show()

From the graph we see that when the true values are non-negative (as is the case in our data), we have the following:
* SMAPE punishes underpredictions more than over predictions. 
* As the prediction gets closer closer to 0, the SMAPE error approaches 1. 
* For over predicitions, as the size of the prediction gets really large, the SMAPE error again approaches 1.

We use these properties to guide us towards finding some transformation functions z = f(y) that will let an L2 regression for target z values more closely emulate minimizing SMAPE.

# Get the Data

In [ ]:
all_df = pd.read_csv('../input/train_1.csv')
all_df.shape

For this notebook, we wish to concentrate on just getting better regression results. So we will simply set every NaN value to 0.

In [ ]:
# Set all NaN to 0.

all_df.fillna(0, inplace = True)

# Dividing the data

The competition has us predicting values 64 days into the future. To measure the accuracy of our models, we will simply let the last 64 days of our training data be the target values Y to predict using the data from the previous days X. Also, this notebook is meant to be as simple as possible, so we will not look at extracting features from the Page data. So we will only be using the time series data.

In [ ]:
# Separate into training data into features and targets.

futureT = 64
X_all = all_df.drop('Page', axis = 1).values[:, :-futureT]
Y_all = all_df.drop('Page', axis = 1).values[:, -futureT:]

For simplicity, we will split our data into a training set, a validation set, and a test set.

In [ ]:
# First split into test and a combination of training and validation.

X_trainvalid, X_test, Y_trainvalid, Y_test = train_test_split(X_all, Y_all, test_size = 0.33, random_state = 32)

# Now split up the training and validation sets.

X_train, X_valid, Y_train, Y_valid = train_test_split(X_trainvalid, Y_trainvalid, test_size = 0.33, random_state = 35)

print('X_train.shape = ', X_train.shape, '\tX_valid.shape = ', X_valid.shape, '\tX_test.shape = ', X_test.shape)
print('Y_train.shape = ', Y_train.shape, '\tY_valid.shape = ', Y_valid.shape, '\tY_test.shape = ', Y_test.shape)

# Set up SMAPE error function

Now we define a SMAPE error function that we will use through out the rest of the notebook.

In [ ]:
def smape(Y_predict, Y_test):
    result = np.linalg.norm(Y_predict - Y_test, axis = 1)
    result = np.abs(result)
    denom = np.linalg.norm(Y_predict, axis = 1)
    denom += np.linalg.norm(Y_test, axis = 1)
    result /= denom
    result *= 100 * 2
    result = np.mean(result)
    return result

# Normal L2 Regression Benchmark

Here we do a simple L2 Regression using a decision tree on the mean of sample over the entire time history of the time series.

In [ ]:
model = Pipeline([ ('means', FunctionTransformer(lambda X : X.mean(axis = 1, keepdims = True))),
                   ('tree', DecisionTreeRegressor(max_depth = 10)) ])
model.fit(X_trainvalid, Y_trainvalid)
Y_predict = model.predict(X_test)
test_smape = {} # We will be storing all of the results of our tests for comparison in a dictionary.
test_smape['benchmark'] = smape(Y_predict, Y_test)
print('SMAPE = ', test_smape['benchmark'])

# Z Transformation Regressions

Now we use simple functions f(Y) to make new target variables Z = f(Y). Then we do regression using the transformed targets Z. To compute the SMAPE score, we invert to get the predicted Y values and then compute the SMAPE score.

For each function f(Y), we will actually search over a hyper-parameter to minimize the SMAPE score.

## First Z Transformation

To motivate the first Z transformation, we consider the mean of the Y values, Y_mean. We plug it into the formula for each term in SMAPE, and we get an average of |y - Y_mean| / (|y| + Y_mean). Fix Y_mean to be a constant C and consider this to be a function of y, that is f(y) = |y - C| / (|y| + C). Now, we should have non-negative y so should be able to ignore the absolute values in the denominator. Let us also ignore the absolute value in the numerator (you can kind of justify this by the fact that the squared nature of L2 means we should have ignored the absolute value from the very beginning). So we are lead to z = f(y) = (y - C) / (y + C) = 1 - 2C / ( y + C ).

Now, doing L2 regression for this z is the same as doing L2 regression for the slightly different f(y) = 1 / (y + C) (which we probably could have simply guessed from the very beginning). This has the nice property of making smaller values larger and therefore more punishing for differences in small values. Furthermore, f(y) goes to zero as y gets really large (the main point being it approaches a bounded value).

In [ ]:
# definition of ztransformation.

def ztransform1(Y, param):
    return 1 / (param + Y)

# inverse transformation, Y = inverseZ(Z)

def inverseZ1(Z, param):
    return -param + 1 / Z

Now let us try searching for the optimal value of C to minimize the SMAPE score when doing L2 regression on Z. We will do a simple grid search.

Now, sometimes our fit will give a predicted value of 0 while the true value is 0. Our SMAPE function is simple and so it will return a NaN value. Let's bypass this by adding a small value of epsilon to the predicted Y values that will not affect the SMAPE score that much.

In [ ]:
# Values to try for param.

param_search = np.arange(20, 420, 20)

# To record results of fits

smapes = []
epsilon = 1e-6

for param in param_search:
    Z_train = ztransform1(Y_train, param)
    model.fit(X_train, Z_train)
    Z_predict = model.predict(X_valid)
    Y_predict = inverseZ1(Z_predict, param)
    newsmape = smape(epsilon + Y_predict, Y_valid)
    smapes.append(newsmape)
    print('param = ', param, ' smape = ', newsmape, ',\t', end = '')
    
plt.plot(param_search, smapes)
plt.title('SMAPES vs param values for Regression on ztransform1')
plt.show()
    

Now let's do a test for param = 240. We will train on all of the training and validation data.

In [ ]:
param = 240
Z_trainvalid = ztransform1(Y_trainvalid, param)
model.fit(X_trainvalid, Z_trainvalid)
Z_predict = model.predict(X_test)
Y_predict = inverseZ1(Z_predict, param)
test_smape['z1'] = smape(epsilon + Y_predict, Y_test)
print('SMAPE = ', test_smape['z1'])

## Second Z Transformation

We justify this one by putting an upper bound on a SMAPE term |p - t| / (|p| + |t|). For simplicity, consider p and t non-negative, so we have |p - t| / (p + t). Now, one can write p - t = (sqrt(p) - sqrt(t)) \* (sqrt(p) + sqrt(t)) and then use the Cauchy-Schwarz inequality on this to give the following upper bound on the original expression in p and t: sqrt(2) * ( sqrt(p) - sqrt(t) ) / sqrt( p + t ).

Similarly to before, we are led to f(y) = (sqrt(y) - np.sqrt(C)) / sqrt(y + C). After some algebra and playing with the possibilities of plus and minus, the inverse function is given by y = C \* ( -1 + z \* sqrt(2 - z\*\*2 ) )\*\*2 / (z\*\*2 - 1)\*\*2. Let's double check this using some graphs.

Now, for non-negative y values, f(y) is between -1 and 1, so the inverse function is always defined. However, we could possibly predict a value of y that is outside the definition of the inverse function. To prevent this, we first force z to be between -1 + epsilon and 1 - epsilon.

In [ ]:
def ztransform2(Y, param):
    return (np.sqrt(Y) - np.sqrt(param)) / np.sqrt(Y + param)

def inverseZ2(Z, param):
    Z2 = np.minimum(Z, 1 - epsilon)
    Z2 = np.maximum(Z2, -1 + epsilon)
    result = -1 - Z2 * np.sqrt(2 - Z2**2)
    result = result / (Z2**2 - 1)
    result = param * result**2
    return result

# Plot ztransform2 for param = 2.0.

plt.figure(figsize = (15, 5))
plt.subplot(121)
domain = np.arange(0, 10, 0.1)
plt.plot(domain, ztransform2(domain, 2.0))
plt.title('ztransform2 for param = 2.0')

# Graph inverseZ2 for param = 1.0 and reflection of graph of ztransform2.

plt.subplot(122)
domain2 = np.arange(-0.7, 0.7, 0.1)
plt.plot(domain2, inverseZ2(domain2, 2.0))
plt.plot(ztransform2(domain, 2.0), domain, color = 'red')
plt.title('Graph of inverseZ2 and reflection of graph of ztransform2')
plt.legend(['inverseZ2','reflection ztransform2'])
plt.show()

Now let's search over param for the optimal value to minimize SMAPE for our model.

In [ ]:
# Values to try for param.

param_search = np.arange(100, 2500, 100)

# To record results of fits

smapes = []
epsilon = 1e-6

for param in param_search:
    Z_train = ztransform2(Y_train, param)
    model.fit(X_train, Z_train)
    Z_predict = model.predict(X_valid)
    Y_predict = inverseZ2(Z_predict, param)
    newsmape = smape(epsilon + Y_predict, Y_valid)
    smapes.append(newsmape)
    print('param = ', param, ' smape = ', newsmape, ',\t', end = '')
    
plt.plot(param_search, smapes)
plt.title('SMAPES vs param values for Regression on ztransform2')
plt.show()
    

Now let's do a test with param = 1000. We will fit the model on both the training and validation data.

In [ ]:
param = 1000
Z_trainvalid = ztransform2(Y_trainvalid, param)
model.fit(X_trainvalid, Z_trainvalid)
Z_predict = model.predict(X_test)
Y_predict = inverseZ2(Z_predict, param)
test_smape['z2'] = smape(epsilon + Y_predict, Y_test)
print('SMAPE = ', test_smape['z2'])

# Review of Test Results

Here we give a review of the test results for our z transformations.

In [ ]:
# Code to generate simple table using notebook output.

description = ['None (Benchmark)', 'Z Transformation 1', 'Z Transformation 2']
keys = ['benchmark', 'z1', 'z2']
scores = [test_smape[key] for key in keys]
output = [x for x in zip(description, scores)]

output

So we can see that there is some improvement in the SMAPE scores when using the Z transformations.